In [1]:
%reload_ext autoreload
%autoreload 2

# Import libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from pylab import rcParams
import json
import os, sys
import string
import logging
import re
import copy
import spacy
import warnings
from spacy import displacy
warnings.filterwarnings("ignore")

cwd = os.getcwd()
frameworkDir = os.path.abspath(os.path.join(cwd, os.pardir, 'src'))
sys.path.append(frameworkDir)

from dackar.workflows.WorkOrderProcessing import WorkOrderProcessing
from dackar.utils.nlp.nlp_utils import generatePatternList
from dackar.utils.opm.OPLparser import OPMobject


logging.basicConfig(format='%(asctime)s %(name)-20s %(levelname)-8s %(message)s', datefmt='%d-%b-%y %H:%M:%S', level=logging.INFO)
logging.getLogger().setLevel(logging.ERROR)

Warming up PyWSD (takes ~10 secs)... took 4.378537178039551 secs.


In [2]:
# Specify Entities Labels and IDs
entLabel = "cws_component"        # user defined entity label
entId = "OPM"  
# Load language model
nlp = spacy.load("en_core_web_lg", exclude=[])
matcher = WorkOrderProcessing(nlp, entID=entId)

entIDList = ['1-91120-P1', '1-91120-PM1', '91120']
patternsEnts = generatePatternList(entIDList, label=entLabel,    id=entId,    nlp=nlp, attr="LEMMA")
matcher.addEntityPattern('cws_entity_ruler', patternsEnts)

text="1-91120-P1, CLEAN PUMPHOUSE. 1-91120-PM1 REQUIRES OIL. 91120, CLEAN TRASH RACKS"

doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)

In [3]:
matcher.reset()
sents = list(text.split('.'))
for sent in sents:
    matcher(sent)
matcher._entStatus

,entity,alias,entity_text,status,conjecture,negation,negation_text
0,1-91120-P1,unit 1 pump,unit 1 pump,CLEAN PUMPHOUSE,False,False,
1,1-91120-PM1,unit 1 pump motor,unit 1 pump motor,OIL,False,False,
2,91120,pump,pump,CLEAN TRASH RACKS,False,False,


In [4]:
for ent in doc.ents:
    print(ent.text, ent._.alias, ent.ent_id_, ent.label_)

1-91120-P1 unit 1 pump OPM cws_component
1-91120-PM1 unit 1 pump motor OPM cws_component
91120 pump OPM cws_component
